# Simulating Planetary Orbits

According to Isaac Newton, the force of gravitational attraction between two objects is given by:

$F = G \frac{m_1m_2}{d^2}$


where:
- G is the gravitational constant (SI Units: $6.67428 \times 10^{-11} N(m/kg)^2$)
- m_1 and m_2 are the masses of the two objects (kilograms)
- d is the distance between the two objects (meters)
- F is the force (Newtons)

Using this equation, Newton determined a formula for calculating how long it took an object to complete an orbit around a central mass. However, when dealing with three or more objects, it’s generally not possible to find a tidy formula to calculate what the three bodies will do.

Instead, such problems are tackled by numeric integration, a brute-force approach where you take all the object positions and velocities at time T, calculate the forces they exert on each other, update the velocities, and calculate the new positions at time T+\epsilon. Then you repeat this in a loop, stepping forward through time, and output or plot the results.

# Approach

To implement this in Python, we’ll use the turtle module to provide a graphical display, subclassing the Turtle class to create a Body class that will have additional attributes: mass for the object’s mass, vx and vy for its velocity, and px and py for its position.

An added method on Body, attraction(), will take another Body instance and return the X and Y components of the force exerted by the other body.

# Solution

In [7]:
# Import Packages
import math
from turtle import *

In [8]:
# The gravitational constant G
G = 6.67428e-11

# Assumed scale: 100 pixels = 1AU.
AU = (149.6e6 * 1000)     # 149.6 million km, in meters.
SCALE = 250 / AU

In [9]:
class Body(Turtle):
    """Subclass of Turtle graphically representing a gravitationally-acting body in 2D.

    Extra attributes:
    mass : mass in kg
    vx, vy: x, y velocities in m/s
    px, py: x, y positions in m
    """
    
    name = 'Body'
    mass = None
    vx = vy = 0.0
    px = py = 0.0
    
    def attraction(self, other):
        """(Body): (fx, fy)

        Returns the force exerted upon this body by the other body.
        """
        # Report an error if the other object is the same as this one.
        if self is other:
            raise ValueError("Attraction of object %r to itself requested"
                             % self.name)

        # Compute the distance of the other body.
        sx, sy = self.px, self.py
        ox, oy = other.px, other.py
        dx = (ox-sx)
        dy = (oy-sy)
        d = math.sqrt(dx**2 + dy**2)

        # Report an error if the distance is zero; otherwise we'll
        # get a ZeroDivisionError exception further down.
        if d == 0:
            raise ValueError("Collision between objects %r and %r"
                             % (self.name, other.name))

        # Compute the force of attraction
        f = G * self.mass * other.mass / (d**2)

        # Compute the direction of the force.
        theta = math.atan2(dy, dx)
        fx = math.cos(theta) * f
        fy = math.sin(theta) * f
        return fx, fy

In [10]:
def update_info(step, bodies):
    """(int, [Body])
    
    Displays information about the status of the simulation.
    """
    print('Step #{}'.format(step))
    for body in bodies:
        s = '{:<8}  Pos.={:>6.2f} {:>6.2f} Vel.={:>10.3f} {:>10.3f}'.format(
            body.name, body.px/AU, body.py/AU, body.vx, body.vy)
        print(s)
    print()

In [11]:
def loop(bodies):
    """([Body])

    Never returns; loops through the simulation, updating the
    positions of all the provided bodies.
    """
    timestep = 24*3600  # One day
    
    for body in bodies:
        body.penup()
        body.hideturtle()

    step = 1
    while step < 730:
        update_info(step, bodies)
        step += 1

        force = {}
        for body in bodies:
            # Add up all of the forces exerted on 'body'.
            total_fx = total_fy = 0.0
            for other in bodies:
                # Don't calculate the body's attraction to itself
                if body is other:
                    continue
                fx, fy = body.attraction(other)
                total_fx += fx
                total_fy += fy

            # Record the total force exerted.
            force[body] = (total_fx, total_fy)

        # Update velocities based upon on the force.
        for body in bodies:
            fx, fy = force[body]
            body.vx += fx / body.mass * timestep
            body.vy += fy / body.mass * timestep

            # Update positions
            body.px += body.vx * timestep
            body.py += body.vy * timestep
            body.goto(body.px*SCALE, body.py*SCALE)
            body.dot(3)

In [6]:
sun = Body()
sun.name = 'Sun'
sun.mass = 1.98892 * 10**30
sun.pencolor('yellow')

earth = Body()
earth.name = 'Earth'
earth.mass = 5.9742 * 10**24
earth.px = -1*AU
earth.vy = 29.783 * 1000            # 29.783 km/sec
earth.pencolor('blue')

# Venus parameters taken from
# http://nssdc.gsfc.nasa.gov/planetary/factsheet/venusfact.html
venus = Body()
venus.name = 'Venus'
venus.mass = 4.8685 * 10**24
venus.px = 0.723 * AU
venus.vy = -35.02 * 1000
venus.pencolor('red')

loop([sun, earth, venus])

Step #1
Sun       Pos.=  0.00   0.00 Vel.=     0.000      0.000
Earth     Pos.= -1.00   0.00 Vel.=     0.000  29783.000
Venus     Pos.=  0.72   0.00 Vel.=     0.000 -35020.000

Step #2
Sun       Pos.=  0.00   0.00 Vel.=     0.001      0.000
Earth     Pos.= -1.00   0.02 Vel.=   512.475  29783.000
Venus     Pos.=  0.72  -0.02 Vel.=  -980.383 -35020.000

Step #3
Sun       Pos.=  0.00  -0.00 Vel.=     0.002     -0.000
Earth     Pos.= -1.00   0.03 Vel.=  1025.026  29774.181
Venus     Pos.=  0.72  -0.04 Vel.= -1961.151 -34992.542

Step #4
Sun       Pos.=  0.00  -0.00 Vel.=     0.003     -0.000
Earth     Pos.= -1.00   0.05 Vel.=  1537.501  29756.538
Venus     Pos.=  0.72  -0.06 Vel.= -2941.533 -34937.583

Step #5
Sun       Pos.=  0.00  -0.00 Vel.=     0.003     -0.000
Earth     Pos.= -1.00   0.07 Vel.=  2049.748  29730.068
Venus     Pos.=  0.72  -0.08 Vel.= -3920.756 -34855.102

Step #6
Sun       Pos.=  0.00  -0.00 Vel.=     0.004     -0.000
Earth     Pos.= -1.00   0.09 Vel.=  2561.615  29694

Step #48
Sun       Pos.=  0.00  -0.00 Vel.=     0.019     -0.037
Earth     Pos.= -0.68   0.72 Vel.= 21735.130  20637.392
Venus     Pos.=  0.17  -0.69 Vel.=-34454.639  -8605.438

Step #49
Sun       Pos.=  0.00  -0.00 Vel.=     0.019     -0.038
Earth     Pos.= -0.67   0.73 Vel.= 22091.513  20260.104
Venus     Pos.=  0.15  -0.70 Vel.=-34688.656  -7624.904

Step #50
Sun       Pos.=  0.00  -0.00 Vel.=     0.018     -0.040
Earth     Pos.= -0.66   0.75 Vel.= 22441.342  19876.583
Venus     Pos.=  0.13  -0.70 Vel.=-34894.411  -6637.843

Step #51
Sun       Pos.=  0.00  -0.00 Vel.=     0.017     -0.041
Earth     Pos.= -0.64   0.76 Vel.= 22784.505  19486.945
Venus     Pos.=  0.11  -0.71 Vel.=-35071.713  -5645.091

Step #52
Sun       Pos.=  0.00  -0.00 Vel.=     0.017     -0.042
Earth     Pos.= -0.63   0.77 Vel.= 23120.893  19091.306
Venus     Pos.=  0.08  -0.71 Vel.=-35220.395  -4647.489

Step #53
Sun       Pos.=  0.00  -0.00 Vel.=     0.016     -0.043
Earth     Pos.= -0.62   0.78 Vel.= 23450.399 

Step #96
Sun       Pos.=  0.00  -0.00 Vel.=    -0.052     -0.067
Earth     Pos.=  0.08   0.99 Vel.= 29968.414  -2192.981
Venus     Pos.= -0.65  -0.30 Vel.=-15508.649  31679.575

Step #97
Sun       Pos.=  0.00  -0.00 Vel.=    -0.054     -0.067
Earth     Pos.=  0.10   0.99 Vel.= 29925.611  -2712.831
Venus     Pos.= -0.66  -0.28 Vel.=-14604.753  32092.743

Step #98
Sun       Pos.=  0.00  -0.00 Vel.=    -0.056     -0.066
Earth     Pos.=  0.12   0.98 Vel.= 29873.714  -3231.843
Venus     Pos.= -0.67  -0.26 Vel.=-13690.155  32479.847

Step #99
Sun       Pos.=  0.00  -0.00 Vel.=    -0.058     -0.065
Earth     Pos.=  0.13   0.98 Vel.= 29812.741  -3749.855
Venus     Pos.= -0.68  -0.24 Vel.=-12765.624  32840.627

Step #100
Sun       Pos.= -0.00  -0.00 Vel.=    -0.060     -0.065
Earth     Pos.=  0.15   0.98 Vel.= 29742.709  -4266.705
Venus     Pos.= -0.68  -0.22 Vel.=-11831.929  33174.849

Step #101
Sun       Pos.= -0.00  -0.00 Vel.=    -0.062     -0.064
Earth     Pos.=  0.17   0.98 Vel.= 29663.64

Step #142
Sun       Pos.= -0.00  -0.00 Vel.=    -0.121      0.015
Earth     Pos.=  0.77   0.63 Vel.= 19304.095 -22915.956
Venus     Pos.= -0.48   0.55 Vel.= 25604.084  23408.018

Step #143
Sun       Pos.= -0.00  -0.00 Vel.=    -0.121      0.018
Earth     Pos.=  0.78   0.62 Vel.= 18902.488 -23244.022
Venus     Pos.= -0.46   0.56 Vel.= 26236.002  22683.261

Step #144
Sun       Pos.= -0.00  -0.00 Vel.=    -0.121      0.021
Earth     Pos.=  0.79   0.60 Vel.= 18495.324 -23564.976
Venus     Pos.= -0.45   0.58 Vel.= 26847.481  21941.833

Step #145
Sun       Pos.= -0.00  -0.00 Vel.=    -0.121      0.024
Earth     Pos.=  0.80   0.59 Vel.= 18082.732 -23878.726
Venus     Pos.= -0.43   0.59 Vel.= 27438.104  21184.303

Step #146
Sun       Pos.= -0.00  -0.00 Vel.=    -0.122      0.026
Earth     Pos.=  0.81   0.57 Vel.= 17664.842 -24185.179
Venus     Pos.= -0.42   0.60 Vel.= 28007.467  20411.253

Step #147
Sun       Pos.= -0.00  -0.00 Vel.=    -0.122      0.029
Earth     Pos.=  0.82   0.56 Vel.= 1724

Step #188
Sun       Pos.= -0.00  -0.00 Vel.=    -0.065      0.136
Earth     Pos.=  0.99  -0.11 Vel.= -2786.922 -29644.640
Venus     Pos.=  0.37   0.63 Vel.= 29843.326 -17527.573

Step #189
Sun       Pos.= -0.00  -0.00 Vel.=    -0.062      0.138
Earth     Pos.=  0.99  -0.13 Vel.= -3295.831 -29587.956
Venus     Pos.=  0.39   0.62 Vel.= 29352.614 -18349.792

Step #190
Sun       Pos.= -0.00  -0.00 Vel.=    -0.059      0.139
Earth     Pos.=  0.99  -0.14 Vel.= -3803.542 -29522.555
Venus     Pos.=  0.41   0.61 Vel.= 28839.421 -19158.601

Step #191
Sun       Pos.= -0.00  -0.00 Vel.=    -0.056      0.141
Earth     Pos.=  0.99  -0.16 Vel.= -4309.904 -29448.463
Venus     Pos.=  0.42   0.60 Vel.= 28304.096 -19953.396

Step #192
Sun       Pos.= -0.00  -0.00 Vel.=    -0.053      0.143
Earth     Pos.=  0.98  -0.18 Vel.= -4814.771 -29365.710
Venus     Pos.=  0.44   0.58 Vel.= 27747.002 -20733.579

Step #193
Sun       Pos.= -0.00  -0.00 Vel.=    -0.051      0.144
Earth     Pos.=  0.98  -0.20 Vel.= -531

...

Step #510
Sun       Pos.= -0.00   0.00 Vel.=     0.032      0.064
Earth     Pos.=  0.80   0.59 Vel.= 18124.238 -23846.783
Venus     Pos.= -0.10  -0.71 Vel.=-35229.818   4615.842

Step #511
Sun       Pos.= -0.00   0.00 Vel.=     0.033      0.063
Earth     Pos.=  0.81   0.58 Vel.= 17706.888 -24153.953
Venus     Pos.= -0.12  -0.70 Vel.=-35082.984   5614.052

Step #512
Sun       Pos.= -0.00   0.00 Vel.=     0.034      0.061
Earth     Pos.=  0.82   0.56 Vel.= 17284.358 -24453.746
Venus     Pos.= -0.14  -0.70 Vel.=-34907.492   6607.520

Step #513
Sun       Pos.= -0.00   0.00 Vel.=     0.034      0.059
Earth     Pos.=  0.83   0.55 Vel.= 16856.783 -24746.078
Venus     Pos.= -0.16  -0.69 Vel.=-34703.501   7595.401

Step #514
Sun       Pos.= -0.00   0.00 Vel.=     0.035      0.058
Earth     Pos.=  0.84   0.53 Vel.= 16424.295 -25030.865
Venus     Pos.= -0.18  -0.69 Vel.=-34471.197   8576.858

Step #515
Sun       Pos.= -0.00   0.00 Vel.=     0.036      0.056
Earth     Pos.=  0.85   0.52 Vel.= 1598

Step #556
Sun       Pos.= -0.00   0.00 Vel.=     0.025      0.007
Earth     Pos.=  0.99  -0.16 Vel.= -4258.055 -29455.415
Venus     Pos.= -0.72  -0.08 Vel.= -4904.092  34778.958

Step #557
Sun       Pos.= -0.00   0.00 Vel.=     0.024      0.007
Earth     Pos.=  0.99  -0.18 Vel.= -4763.050 -29373.552
Venus     Pos.= -0.72  -0.06 Vel.= -3924.895  34889.304

Step #558
Sun       Pos.= -0.00   0.00 Vel.=     0.023      0.006
Earth     Pos.=  0.98  -0.19 Vel.= -5266.419 -29283.058
Venus     Pos.= -0.72  -0.04 Vel.= -2943.748  34972.021

Step #559
Sun       Pos.= -0.00   0.00 Vel.=     0.022      0.006
Earth     Pos.=  0.98  -0.21 Vel.= -5768.013 -29183.967
Venus     Pos.= -0.72  -0.02 Vel.= -1961.434  35027.107

Step #560
Sun       Pos.= -0.00   0.00 Vel.=     0.021      0.005
Earth     Pos.=  0.98  -0.23 Vel.= -6267.687 -29076.315
Venus     Pos.= -0.72   0.00 Vel.=  -978.730  35054.583

Step #561
Sun       Pos.= -0.00   0.00 Vel.=     0.020      0.005
Earth     Pos.=  0.97  -0.24 Vel.= -676

Step #602
Sun       Pos.= -0.00   0.00 Vel.=    -0.006      0.021
Earth     Pos.=  0.59  -0.82 Vel.=-23754.627 -17643.707
Venus     Pos.= -0.29   0.67 Vel.= 31445.239  14439.993

Step #603
Sun       Pos.= -0.00   0.00 Vel.=    -0.006      0.022
Earth     Pos.=  0.58  -0.83 Vel.=-24050.562 -17233.558
Venus     Pos.= -0.27   0.68 Vel.= 31827.128  13562.757

Step #604
Sun       Pos.= -0.00   0.00 Vel.=    -0.006      0.023
Earth     Pos.=  0.56  -0.84 Vel.=-24339.442 -16818.519
Venus     Pos.= -0.26   0.69 Vel.= 32184.814  12675.709

Step #605
Sun       Pos.= -0.00   0.00 Vel.=    -0.006      0.024
Earth     Pos.=  0.55  -0.85 Vel.=-24621.187 -16398.709
Venus     Pos.= -0.24   0.69 Vel.= 32518.062  11779.503

Step #606
Sun       Pos.= -0.00   0.00 Vel.=    -0.006      0.025
Earth     Pos.=  0.53  -0.85 Vel.=-24895.723 -15974.251
Venus     Pos.= -0.22   0.70 Vel.= 32826.650  10874.795

Step #607
Sun       Pos.= -0.00   0.00 Vel.=    -0.005      0.026
Earth     Pos.=  0.52  -0.86 Vel.=-2516

Step #648
Sun       Pos.= -0.00   0.00 Vel.=     0.031      0.054
Earth     Pos.= -0.15  -1.00 Vel.=-29221.332   4310.777
Venus     Pos.=  0.55   0.48 Vel.= 23130.668 -25869.021

Step #649
Sun       Pos.= -0.00   0.00 Vel.=     0.033      0.054
Earth     Pos.= -0.17  -0.99 Vel.=-29144.066   4808.854
Venus     Pos.=  0.56   0.47 Vel.= 22410.749 -26508.101

Step #650
Sun       Pos.= -0.00   0.00 Vel.=     0.034      0.054
Earth     Pos.= -0.19  -0.99 Vel.=-29058.382   5305.573
Venus     Pos.=  0.57   0.45 Vel.= 21673.121 -27127.512

Step #651
Sun       Pos.= -0.00   0.00 Vel.=     0.036      0.054
Earth     Pos.= -0.20  -0.99 Vel.=-28964.302   5800.793
Venus     Pos.=  0.58   0.44 Vel.= 20918.300 -27726.762

Step #652
Sun       Pos.= -0.00   0.00 Vel.=     0.037      0.054
Earth     Pos.= -0.22  -0.98 Vel.=-28861.851   6294.375
Venus     Pos.=  0.59   0.42 Vel.= 20146.817 -28305.368

Step #653
Sun       Pos.= -0.00   0.00 Vel.=     0.039      0.054
Earth     Pos.= -0.24  -0.98 Vel.=-2875

Step #694
Sun       Pos.= -0.00   0.00 Vel.=     0.099      0.006
Earth     Pos.= -0.81  -0.60 Vel.=-17659.638  23796.290
Venus     Pos.=  0.61  -0.38 Vel.=-18634.933 -29964.056

Step #695
Sun       Pos.= -0.00   0.00 Vel.=     0.100      0.003
Earth     Pos.= -0.82  -0.58 Vel.=-17251.713  24097.918
Venus     Pos.=  0.60  -0.40 Vel.=-19479.058 -29436.651

Step #696
Sun       Pos.= -0.00   0.00 Vel.=     0.100      0.001
Earth     Pos.= -0.83  -0.57 Vel.=-16838.614  24392.624
Venus     Pos.=  0.59  -0.41 Vel.=-20308.419 -28885.103

Step #697
Sun       Pos.= -0.00   0.00 Vel.=     0.101     -0.001
Earth     Pos.= -0.84  -0.56 Vel.=-16420.456  24680.318
Venus     Pos.=  0.57  -0.43 Vel.=-21122.307 -28309.817

Step #698
Sun       Pos.= -0.00   0.00 Vel.=     0.102     -0.003
Earth     Pos.= -0.85  -0.54 Vel.=-15997.355  24960.913
Venus     Pos.=  0.56  -0.45 Vel.=-21920.020 -27711.219

Step #699
Sun       Pos.= -0.00   0.00 Vel.=     0.102     -0.006
Earth     Pos.= -0.86  -0.53 Vel.=-1556

# Code Discussion

The system described in the code consists of the Sun, Earth, and Venus, so the main() function creates three Body instances for each body and passed to the loop() function.

The loop() function is the heart of the simulation, taking a list of Body instances and then performing simulation steps forever. The time step chosen is one day, which works well for our Sun/Earth/Venus example. When you run the program, you can see how long it takes for the plot to complete an entire orbit; for Earth it’s the expected 365 days and for Venus it’s 224 days.

# Lessons Learned

Each simulation step requires calculating $N * (N-1)$ distances and attractions, so the time complexity is $O(N^2)$. On a laptop or desktop, the display will be visible changing up to around 20 objects. More efficient coding would let us handle more objects; we could rewrite the calculations in C or parallelize the code to divide the work in each step among multiple threads or CPUs. You could also adjust the timestep dynamically: if objects are far apart, a larger timestep would introduce less error, and the timestep could be shortened when objects are interacting more closely.

These techniques would increase our practical limit to hundreds $(10^3)$ or thousands $(10^4)$ of objects, but this means we can’t simulate even a small galaxy, which might contain tens of millions of stars $(10^7)$. (Our galaxy is estimated to have around 200 billion stars, $2 \times 10^{11}$.) Entirely different approaches need to be taken for that problem size; for example, the attraction of distant particles is approximated and only nearby particles are calculated exactly. The references include a survey by Drs. Trenti and Hut that describes the techniques used for larger simulations.

# References

- http://www.scholarpedia.org/article/N-body_simulations This survey, by Dr. Michele Trenti and Dr. Piet Hut, describes how the serious scientific N-body simulators work, using trees to approximate the attraction at great distances. Such programs are able to run in O(N log(N)) time.
- http://ssd.jpl.nasa.gov/horizons.cgi NASA’s Jet Propulsion Laboratory provides a system called HORIZONS that returns accurate positions and velocities for objects within the solar system. In the example code, the values used are only rough approximations; the orbital distances and planet velocities are set to the mean distances and their relative positions don’t correspond to any actual point in time – but they produce reasonable output. 
- https://fiftyexamples.readthedocs.io/en/latest/gravity.html Original post of discussion and code